In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras 
from tqdm import tqdm
AUTOTUNE = tf.data.experimental.AUTOTUNE
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pathlib
import datetime
import time
from google.colab import files
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from google.colab import drive

#drive.mount('/content/drive')

In [0]:

train_data_dir= '/content/drive/My Drive/images_dataset'
CATEGORIES  = ['MEL_cropped','BKL_cropped', 'BCC_cropped']
IMG_SIZE = 80

training_data = []

def create_training_data():
  # this function takes the images and makes them into an array of np arrays and an array of labels
  for category in CATEGORIES:
    path = os.path.join(train_data_dir,category)
    class_num = CATEGORIES.index(category)
    for img in tqdm(os.listdir(path)):
      img_array = cv2.imread(os.path.join(path,img))  # convert to array
      new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
      training_data.append([new_array, class_num])
      

create_training_data()

import random

random.shuffle(training_data)

X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)


In [0]:
X = np.array(X/255.0)#normalisation
y=np.array(y)

batch_size = 20

dense_layers = [3,4,5] 
conv_layer_sizes = [32]
dense_layer_sizes=[32]
conv_layers = [3]

for dense_layer in dense_layers:
    for conv_layer_size in conv_layer_sizes:
      for dense_layer_size in dense_layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-conv_nodes-{}-nodes-{}-dense-{}".format(conv_layer, conv_layer_size, dense_layer_size, dense_layer, int(time.time()))
            print(NAME)
            #tf.reset_default_graph()
            model = Sequential()

            model.add(Conv2D(conv_layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                  model.add(Conv2D(conv_layer_size, (3, 3)))
                  model.add(Activation('relu'))
                  model.add(MaxPooling2D(pool_size=(2, 2)))
            
            model.add(Flatten())
            for _ in range(dense_layer):
                model.add(Dense(dense_layer_size))
                model.add(Activation('relu'))


            model.add(Dense(3))
            model.add(Activation('softmax'))

            model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
                          optimizer='adam',
                          metrics=['accuracy'])
            
            #model.summary()
            logdir = "logs/{}".format(NAME)
            tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

            model.fit(X, y,
                        batch_size=batch_size,
                        epochs=10,
                        validation_split=0.1,
                        callbacks=[tensorboard_callback])
        

            


In [0]:
!zip -r /content/file.zip /content/logs


In [0]:
files.download("/content/file.zip")

In [0]:
%load_ext tensorboard

In [0]:
%tensorboard --logdir logs

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
#this part of the script is used to preprocess the validation dataset

test_data_dir= '/content/drive/My Drive/images_dataset/Test_Dataset'
CATEGORIES  = ['MEL','BKL', 'BCC']
IMG_SIZE = 80

testing_data = []

def create_testing_data():
  for category in CATEGORIES:
    path = os.path.join(test_data_dir,category)
    class_num = CATEGORIES.index(category)
    for img in tqdm(os.listdir(path)):
      img_array = cv2.imread(os.path.join(path,img))  # convert to array
      new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
      testing_data.append([new_array, class_num])
      

create_testing_data()

import random

random.shuffle(testing_data)

X_test = []
y_test = []

for features,label in testing_data:
    X_test.append(features)
    y_test.append(label)

X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y_test=np.array(y_test)

In [0]:
model.evaluate(X_test,y_test,batch_size=20)